# Make figures for paper on geometric images

## Authors:
- **David W. Hogg** (NYU) (MPIA) (Flatiron)
- **Soledad Villar** (JHU)

## To-do items and bugs:
- Plot filters in groups with subplots or equivalent (ie, not just one filter per plot).
- Figure out how to plot 2-tensor filters.
- Figure out which cases need to be plotted.
- Figure out a better way to plot in `D=3`.

In [ ]:
import itertools as it
import numpy as np
import geometric as geom
import finufft
import pylab as plt
%load_ext autoreload
%autoreload 2

In [ ]:
D = 2
group_operators = geom.make_all_operators(D)

In [ ]:
geom.test_group(group_operators)

In [ ]:
filters = {}
names = {}
maxn = {}
for M in [3, 5]:
    maxn[(D, M)] = 0
    for k, parity in it.product((0, 1), (1, -1)):
        key = (D, M, k, parity)
        filters[key] = geom.get_unique_invariant_filters(M, k, parity, D, group_operators)
        n = len(filters[key])
        if n > maxn[(D, M)]:
            maxn[(D, M)] = n
        names[key] = ["{} {}".format(geom.ktensor.name(k, parity), i) for i in range(n)]

In [ ]:
dpi = 300
paritysign = {1: "+", -1: "-"}
for key in filters.keys():
    D, M, k, parity = key
    fig = geom.plot_filters(filters[key], names[key], maxn[(D, M)])
    plotname = "filter{}{}_{}_{}.png".format(paritysign[parity], k, D, M)
    fig.savefig(plotname, dpi=dpi)

In [ ]:
# make a sensible smooth scalar image on a 2-torus
N = 16
D = 2
np.random.seed(42)
image = np.random.normal(size=D * (N, ))
foo = np.pi * np.arange(-1. + 1. / N, 1., 2. / N)
ys, xs = np.meshgrid(foo, foo) # ys, xs or xs, ys??
ft = finufft.nufft2d1(xs.flatten(), ys.flatten(), image.flatten().astype(complex), (6, 5))
package = finufft.nufft2d2(xs.flatten(), ys.flatten(), ft).reshape(N, N).real
package -= np.mean(package)
package /= np.sqrt(np.mean(package ** 2))
scalar_image = geom.geometric_image(package, 1, D)

In [ ]:
C_s = filters[(D, 5, 0, 1)][4]
C_ps = filters[(D, 5, 0, -1)][0]
C_v = filters[(D, 3, 1, 1)][0]
C_pv = filters[(D, 3, 1, -1)][0]

In [ ]:
sstars = scalar_image.convolve_with(C_s)
sstarps = scalar_image.convolve_with(C_ps)
sstarv = scalar_image.convolve_with(C_v)
sstarpv = scalar_image.convolve_with(C_pv)
monomials = {}
monomials[1] = [(scalar_image, r"s", "s"),
                (sstars,  r"s\ast C_{s}",  "sCs"),
                (sstarps, r"s\ast C_{ps}", "sCps"),
                (sstarv,  r"s\ast C_{v}",  "sCv"),
                (sstarpv, r"s\ast C_{pv}", "sCpv"),
               ]

In [ ]:
degree = 1
for image, latex, name in monomials[degree]:
    ax = geom.plot_image(image)
    plt.title("$" + latex + "$")
    plt.savefig("{}.png".format(name), dpi=dpi)

In [ ]:
geom.plot_scalar_image((sstarv * sstarpv).contract(0, 1),
                       vmin=-1, vmax=1)
plt.title("this should be zero")

In [ ]:
# next order!
monomials[2] = []
n = len(monomials[1])
for i in range(n):
    for j in range(i, n):
        image1, latex1, name1 = monomials[1][i]
        image2, latex2, name2 = monomials[1][j]
        monomials[2] += [(image1 * image2, "(" + latex1 + r")\otimes(" + latex2 + ")", name1 + name2), ]
foo = []
for image, latex, name in monomials[2]:
    if image.k > 1:
        image = image.contract(0, 1)
        latex = r"\mathrm{Tr}(" + latex + ")"
    foo += [(image, latex, name), ]
monomials[2] = foo

In [ ]:
degree = 2
for image, latex, name in monomials[degree]:
    ax = geom.plot_image(image)
    plt.title("$" + latex + "$")
    plt.savefig("{}.png".format(name), dpi=dpi)